# Gene enrichment analysis

In [23]:
from pymodulon.enrichment import *
from pymodulon.data.example_data import load_ecoli_data, trn
ica_data = load_ecoli_data()

## General functions

To perform a basic enrichment test between two gene sets, use the ``compute_enrichment`` function.
Optional arguments:

* ``label``: Label for your target gene set (e.g. regulator name or GO term)

In [21]:
gene_set = ['b0002','b0003','b0004'] # e.g. an iModulon or other gene set
target_genes = ['b0002','b0005','b0007'] # e.g. a regulon or genes in a COG category
all_genes = ica_data.gene_names # List of all genes in your expression dataset

In [22]:
compute_enrichment(gene_set,target_genes,all_genes)

pvalue             0.002293
precision          0.333333
recall             0.333333
f1score            0.333333
TP                 1.000000
target_set_size    3.000000
gene_set_size      3.000000
dtype: float64

To perform an enrichment test against a regulon in a TRN table, use the ``compute_regulon`` function.

In [27]:
compute_regulon_enrichment(gene_set,'lrp',all_genes,trn) 

pvalue             0.000131
precision          0.015000
recall             1.000000
f1score            0.029557
TP                 3.000000
regulon_size     200.000000
gene_set_size      3.000000
n_regs             1.000000
Name: lrp, dtype: float64

Each row of the TRN table represents a regulatory interaction. The table requires the columns ``regulator`` and ``gene_id``.

In [35]:
trn.head()

,regulator,gene_id,effect
0,FMN,b3041,-
1,L-tryptophan,b3708,+
2,L-tryptophan,b3709,+
3,TPP,b0066,-
4,TPP,b0067,-


To perform an enrichment against all regulons in a TRN table, use the ``compute_trn_enrichment`` function. This function includes false detection correction using the Benjamini-Hochberg procedure to compute [Q-values](https://en.wikipedia.org/wiki/Q-value_(statistics)). By default, the false detection rate is ``1e-5``, and can be updated using the ``fdr`` argument.

In [33]:
compute_trn_enrichment(gene_set,all_genes,trn)

,pvalue,precision,recall,f1score,TP,regulon_size,gene_set_size,n_regs,qvalue
thr-tRNA,0.000000e+00,1.000000,1.0,1.000000,3.0,3.0,3.0,1.0,0.000000e+00
ile-tRNA,8.354256e-09,0.333333,1.0,0.500000,3.0,9.0,3.0,1.0,2.923990e-08
lrp,1.306248e-04,0.015000,1.0,0.029557,3.0,200.0,3.0,1.0,3.047911e-04
arcA,1.540883e-03,0.006608,1.0,0.013129,3.0,454.0,3.0,1.0,2.157237e-03
fnr,1.411995e-03,0.006803,1.0,0.013514,3.0,441.0,3.0,1.0,2.157237e-03
crp,3.335561e-03,0.005111,1.0,0.010169,3.0,587.0,3.0,1.0,3.891487e-03


``compute_trn_enrichment`` can perform enrichment against complex regulons in boolean combinations. The ``max_regs`` argument determines the number of regulators to include in the complex regulon, and the ``method`` argument determines how to combine complex regulons. ``method='or'`` computes enrichments against the union of regulons, ``'and'`` computes enrichment against intersection of regulons, and ``'both'`` performs both tests (default).

In [36]:
compute_trn_enrichment(gene_set,all_genes,trn,max_regs=2).head()

,pvalue,precision,recall,f1score,TP,regulon_size,gene_set_size,n_regs,qvalue
fnr+ile-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,0.0
thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,0.0
arcA+ile-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,0.0
lrp+thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,0.0
arcA+thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,0.0


Increasing the number of regulators for complex regulons beyond three can lead to a combinatorial explosion. If you are sure you want to search for larger complex regulons, use ``force=True``

In [38]:
compute_trn_enrichment(gene_set,all_genes,trn,max_regs=3,force=True).head()

,pvalue,precision,recall,f1score,TP,regulon_size,gene_set_size,n_regs,qvalue
arcA+ile-tRNA+lrp,0.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,0.0
arcA+ile-tRNA+thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,0.0
arcA+lrp+thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,0.0
arcA+rpoD+thr-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,0.0
crp+fnr+ile-tRNA,0.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,0.0


## Using the ``IcaData`` object

In [ ]:
# Compare a single iModulon to a simple regulon
ica_data.compute_regulon_enrichment('GlpR',regulator='glpR')

In [ ]:
# Compare a single iModulon to a complex regulon
ica_data.compute_regulon_enrichment('Copper','cusR/cueR')

In [ ]:
# Get all significant enrichments for all iModulons. 
# Optional parameters are same as the global compute_trn_enrichment function
ica_data.compute_trn_enrichment()

### Save to sample table
You can directly save iModulon enrichments to the sample table using either the `ica_data.compute_regulon_enrichment` function or `ica_data.compute_trn_enrichment` function. Columns unrelated to regulon enrichments (e.g. `Function` and `Category`) are left alone.

In [ ]:
ica_data.imodulon_table.loc[['GlpR']]

In [ ]:
ica_data.compute_regulon_enrichment('GlpR',regulator='crp',save=True)
ica_data.imodulon_table.loc[['GlpR']]

In [ ]:
# The enrichment with the lowest qvalue is saved as the official enrichment
ica_data.compute_trn_enrichment(save=True)
ica_data.imodulon_table